### Code Hist.

 - CODE  
    &ensp; : Crawling - 특일 정보 조회 (KASI)

  - DATE  
    &ensp; 2023-11-29 Created  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1)   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2)   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3)   
    
 - DESC  
    &ensp; : 전처리 - 한국지역난방공사 열판매량/열공급량   
    &emsp; 1) 결측치가 없어서, 그대로 사용  
    &emsp;&ensp;&ensp; 
    &emsp;&ensp;&ensp; (Crawl Code 없음)   
    &emsp; 2) 

 - DATA  
    &emsp; <"Input">  
    1) None (Input Dataset)  
    &emsp;- Period :   
    &emsp;- Interval : 

    &emsp; <"Output">  
    1) Hourly (관측소/년도별 출력)  
    &nbsp;df_data_cal.to_csv(data_dir + 'KASI_DATE_D_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    2) Daily (관측소/년도별 출력)  
    &nbsp;df_data_cal_24.to_csv(data_dir + 'KASI_DATE_H_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['locdate', 'YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'HOUR', 'MINUTE']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    

 - Related Link  
    &ensp; : 

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [1]:
#region Basic_Import
## Basic
import os
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
from sklearn.preprocessing import MinMaxScaler, StandardScaler
## PCA
from sklearn.decomposition import PCA

## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score   

## Excel/CSV
import openpyxl, xlrd

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
## Import_Local (현재 프로그램에서 미사용)
from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
from Src_Dev_Common import DEV_Common_Data_Analysis as com_Analysis
from Src_Dev_Common import DEV_Common_Data_Preprocessing as com_Prep
from Src_Dev_Common import DEV_Common_Data_Visualization as com_Visual
from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_ASOS
from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC


### 01-01-02. Config (Directory, Params)

In [4]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [5]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-01-14 23:25:07.709587
2024 / 1 / 14
23 : 25


## 01-02. Data Load (df_raw)

### 01-02-01. KDHC Heat Usage (Intergrated)

In [6]:
# Define data root directory
str_dir_kdhcHeat = "../data/data_Energy_KDHC_National_Heat/"
str_file = "KDHC_COMB_HEAT_2019-2022.csv"
print(os.listdir(str_dir_kdhcHeat))

df_kdhc_heat = pd.read_csv(str_dir_kdhcHeat + str_file, index_col = 0)
df_kdhc_heat['METER_DATE'] = pd.to_datetime(df_kdhc_heat['METER_DATE'])
df_kdhc_heat

['KDHC_COMB_HEAT-CJ_ASOS_KASI_2010-2023.csv', 'KDHC_COMB_HEATSxPCA01_2019-2022_Scaled.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_complete_analysis.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_linearReg.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_mean_imp.csv', 'KDHC_COMB_HEAT_2019-2022.csv', 'KDHC_COMB_KANGNAMHEATxWEATHER_2019-2022.csv', 'KDHC_HEAT_19Region_hr.csv', 'KDHC_HEAT_BUNDANG_2019-2022.csv', 'KDHC_HEAT_CHEONGJU_2010_2021.csv', 'KDHC_HEAT_CHEONGJU_2010_2021_1st_Check.csv', 'KDHC_HEAT_CHEONGJU_2019-2022.csv', 'KDHC_HEAT_DAEGU_2019-2022.csv', 'KDHC_HEAT_DONGTAN_2019-2022.csv', 'KDHC_HEAT_GOYANG_2019-2022.csv', 'KDHC_HEAT_GWANGGYO_2019-2022.csv', 'KDHC_HEAT_GWANGJU_JEONNAM_2019-2022 - 복사본.csv', 'KDHC_HEAT_GWANGJU_JEONNAM_2019-2022.csv', 'KDHC_HEAT_HWASUNG_2019-2022.csv', 'KDHC_HEAT_JOONGANG_2019-2022.csv', 'KDHC_HEAT_KANGNAM_2019-2022.csv', 'KDHC_HEAT_KIMHAE_2019-2022.csv', 'KDHC_HEAT_PAJU_2019-2022.csv', 'KDHC_HEAT_PANGYO_2019-2022.csv', 'KDHC_HEAT_PYO

,METER_DATE,PAJU,GOYANG,SAMSONG,JOONGANG,KANGNAM,PANGYO,BUNDANG,YONGIN,GWANGGYO,SUWON,HWASUNG,DONGTAN,PYONGTAEK,CHEONGJU,SEJONG,KIMHAE,DAEGU,YANGSAN,GWANGJU_JEONNAM
0,2019-10-01 01:00:00,20,73.0000000000,8.0000000000,20.0000000000,61.3000000000,8.0000000000,55.1000000000,30.0000000000,14.0000000000,26.0000000000,11.0000000000,21,0.5000000000,16,18.0000000000,8,19.0000000000,10,3
1,2019-10-01 01:00:00,20,73.0000000000,8.0000000000,20.0000000000,61.3000000000,8.0000000000,55.1000000000,30.0000000000,14.0000000000,26.0000000000,11.0000000000,21,0.5000000000,16,18.0000000000,8,19.0000000000,10,3
2,2019-10-01 02:00:00,15,65.0000000000,12.0000000000,18.0000000000,61.3000000000,4.0000000000,56.0000000000,25.0000000000,7.0000000000,29.0000000000,12.0000000000,16,0.5000000000,13,15.0000000000,7,16.0000000000,9,2
3,2019-10-01 02:00:00,15,65.0000000000,12.0000000000,18.0000000000,61.3000000000,4.0000000000,56.0000000000,25.0000000000,7.0000000000,29.0000000000,12.0000000000,16,0.5000000000,13,15.0000000000,7,16.0000000000,9,2
4,2019-10-01 03:00:00,14,58.0000000000,12.0000000000,17.0000000000,56.3000000000,5.0000000000,51.0000000000,18.0000000000,11.0000000000,33.0000000000,12.0000000000,14,0.5000000000,10,15.0000000000,7,15.0000000000,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27298,2022-09-30 20:00:00,41,81.0000000000,44.0000000000,45.0000000000,120.0000000000,37.0000000000,74.4000000000,87.0000000000,40.0000000000,73.0000000000,39.0000000000,61,12.0000000000,36,44.0000000000,12,30.0000000000,18,6
27299,2022-09-30 21:00:00,40,90.0000000000,46.0000000000,46.0000000000,112.0000000000,36.0000000000,78.4000000000,91.0000000000,25.0000000000,71.0000000000,36.0000000000,55,4.0000000000,30,44.0000000000,11,35.0000000000,22,5
27300,2022-09-30 22:00:00,37,95.0000000000,39.0000000000,44.0000000000,95.0000000000,24.0000000000,65.4000000000,81.0000000000,16.0000000000,65.0000000000,29.0000000000,45,2.0000000000,27,43.0000000000,12,32.0000000000,22,6
27301,2022-09-30 23:00:00,32,98.0000000000,35.0000000000,41.0000000000,89.0000000000,19.0000000000,51.4000000000,80.0000000000,16.0000000000,64.0000000000,25.0000000000,32,3.0000000000,25,40.0000000000,11,34.0000000000,22,6


In [7]:
df_kdhc_heat.columns

Index(['METER_DATE', 'PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG', 'KANGNAM',
       'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG',
       'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG', 'KIMHAE', 'DAEGU',
       'YANGSAN', 'GWANGJU_JEONNAM'],
      dtype='object')

In [8]:
list_tar = ['PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG'
            , 'KANGNAM'
            , 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG', 'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG'
            , 'KIMHAE', 'DAEGU', 'YANGSAN', 'GWANGJU_JEONNAM']

dict_season = {12 : 3, 1 : 3, 2 : 3 ## Winter
               , 3 : 0, 4 : 0, 5 : 0 ## Spring
               , 6 : 1, 7 : 1, 8 : 1 ## Summer
               , 9 : 2, 10 : 2, 11 : 2 ## Autumn
               }

dict_timePeriod = {0 : 0, 1 : 0, 2 : 0, 3 : 0, 4 : 0, 5 : 0 ## Period 0 (0 ~ 5)
                   , 6 : 1, 7 : 1, 8 : 1, 9 : 1, 10 : 1, 11 : 1 ## Period 1 (6 ~ 11)
                   , 12 : 2, 13 : 2, 14 : 2, 15 : 2, 16 : 2, 17 : 2 ## Period 2 (12 ~ 17)
                   , 18 : 3, 19 : 3, 20 : 3, 21 : 3, 22 : 3, 23 : 3 ## Period 3 (18 ~ 23)
                   }

## 'METER_DATE' Column으로부터 YMD 생성
df_kdhc_heat = com_date.create_col_ymdhm(df_kdhc_heat, 'METER_DATE').drop_duplicates()

df_kdhc_heat['SEASON'] = 0
df_kdhc_heat['TIME_PERIOD'] = 0
## MONTH에 따른 SEASON 변수 추가
## HOUR에 따른 Time_Period 변수 추가
for i in range(0, len(df_kdhc_heat)):
    df_kdhc_heat['SEASON'].iloc[i] = dict_season.get(df_kdhc_heat['MONTH'].iloc[i])
    df_kdhc_heat['TIME_PERIOD'].iloc[i] = dict_timePeriod.get(df_kdhc_heat['HOUR'].iloc[i])
    
## 종합된 열 판매량을 지역별로 분리
for col_tar in list_tar:
    df_region = df_kdhc_heat[['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'SEASON'
                              , 'HOUR', 'TIME_PERIOD'
                              , col_tar]]
    df_region.to_csv(str_dir_kdhcHeat + 'KDHC_HEAT_' + str(col_tar) + '_2019-2022.csv')

In [9]:
## 저장된 지역별 열 판매량을 계절/시간대별로 요약
for col_tar in list_tar:
    df_region = pd.read_csv(str_dir_kdhcHeat + 'KDHC_HEAT_' + str(col_tar) + '_2019-2022.csv'
                            , encoding = 'cp949', index_col = 0)
    df_region_summary = pd.DataFrame()
    df_region_summary_daily_sum = df_region.groupby(['YEAR', 'MONTH', 'DAY', 'SEASON', 'TIME_PERIOD'])[col_tar].sum()
    df_region_summary_season_daily_mean = df_region_summary_daily_sum.groupby(['YEAR', 'SEASON', 'TIME_PERIOD']).mean()
    df_region_summary_season_daily_mean.to_csv(str_dir_kdhcHeat + 'KDHC_HEAT_SEASON_TP_MEAN_' + str(col_tar) + '.csv')

# df_region_summary_daily_sum.head(25)
df_region_summary_season_daily_mean.head(25)

YEAR  SEASON  TIME_PERIOD
2019  2       0              60.5409836066
              1              66.6885245902
              2              44.7377049180
              3              65.4426229508
      3       0             137.5806451613
              1             140.5483870968
              2              88.6129032258
              3             135.0645161290
2020  0       0              75.2065217391
              1              87.3478260870
              2              57.8804347826
              3              84.6521739130
      1       0              24.6521739130
              1              57.0434782609
              2              70.7717391304
              3              43.0326086957
      2       0              58.0439560440
              1              69.3076923077
              2              55.1538461538
              3              69.0549450549
      3       0             160.7142857143
              1             158.7692307692
              2             

In [12]:
## 계절/시간대별로 요약된 지역별 열 판매량을 
## SEASON / TIME_PERIOD별로 정렬 및 Transpose
df_region_summary_total = pd.DataFrame()
for col_tar in list_tar:
    df_region = pd.read_csv(str_dir_kdhcHeat + 'KDHC_HEAT_SEASON_TP_MEAN_' + str(col_tar) + '.csv'
                            , encoding = 'cp949') #, index_col = 0)

    df_region['YEAR'] = df_region['YEAR'].astype('string')
    df_region['SEASON'] = df_region['SEASON'].astype('string')
    df_region['TIME_PERIOD'] = df_region['TIME_PERIOD'].astype('string')
    df_region['idx'] = df_region['YEAR'] + '-' + df_region['SEASON'] + '-' + df_region['TIME_PERIOD']

    df_region = df_region[['idx', col_tar]]
    df_region = df_region.set_index('idx')

    df_region = df_region.transpose()
    df_region_summary_total = pd.concat([df_region_summary_total, df_region])

# df_region.head(25)
df_region_summary_total = df_region_summary_total.reset_index()
df_region_summary_total['target'] = 0
df_region_summary_total.to_csv(str_dir_kdhcHeat + 'KDHC_HEAT_tmp.csv')
df_region_summary_total

idx,index,2019-2-0,2019-2-1,2019-2-2,2019-2-3,2019-3-0,2019-3-1,2019-3-2,2019-3-3,2020-0-0,...,2022-1-3,2022-2-0,2022-2-1,2022-2-2,2022-2-3,2022-3-0,2022-3-1,2022-3-2,2022-3-3,target
0,PAJU,257.8688524590,235.3770491803,149.5245901639,301.8852459016,591.3548387097,504.9354838710,285.2580645161,622.0645161290,277.0978260870,...,176.4565217391,40.5806451613,127.5666666667,123.8333333333,199.2000000000,790.0169491525,736.6440677966,535.3220338983,859.9152542373,0
1,GOYANG,1151.2622950820,1143.0819672131,934.9836065574,1522.0819672131,2837.2258064516,2801.8709677419,2069.6774193548,2971.1290322581,1259.3804347826,...,519.9565217391,270.5161290323,458.8666666667,461.6000000000,522.0666666667,3248.1864406780,3473.5423728814,2638.3050847458,3710.9830508475,0
2,SAMSONG,282.2295081967,279.9836065574,230.7049180328,327.6885245902,613.3870967742,589.0322580645,480.1935483871,666.2580645161,308.3478260870,...,323.5163043478,116.2580645161,217.5000000000,270.1666666667,253.9000000000,874.1864406780,851.4576271186,673.4237288136,913.7118644068,0
3,JOONGANG,581.0836065574,604.1852459016,472.6032786885,667.5213114754,1302.5806451613,1437.9354838710,1199.8064516129,1502.1612903226,646.1663043478,...,216.6239130435,124.7096774194,195.9000000000,189.3000000000,211.5666666667,1426.3898305085,1639.8305084746,1376.0508474576,1633.7152542373,0
4,KANGNAM,1370.3754098361,1357.1524590164,1011.1442622951,1488.7704918033,2908.2064516129,2891.8096774194,2347.3193548387,3215.3870967742,1415.0858695652,...,567.0500000000,320.2258064516,565.8666666667,617.1666666667,589.9666666667,3377.6576271186,3503.0542372881,2755.4067796610,3849.1372881356,0
5,PANGYO,163.1639344262,233.8852459016,228.6885245902,228.8360655738,408.5806451613,538.2903225806,460.2258064516,526.4193548387,189.3913043478,...,277.4673913043,62.0645161290,253.7333333333,430.7333333333,201.6666666667,583.3220338983,726.0847457627,581.0169491525,655.2881355932,0
6,BUNDANG,788.8081967213,896.5901639344,651.8180327869,903.3377049180,1665.5774193548,1952.7548387097,1492.6548387097,1872.8645161290,817.4608695652,...,284.6891304348,219.8677419355,346.7666666667,383.5600000000,268.5133333333,2151.4830508475,2416.0440677966,1915.9949152542,2269.0847457627,0
7,YONGIN,1153.2622950820,1152.2622950820,880.3278688525,1296.2131147541,2534.5806451613,2347.5161290323,1751.6451612903,2671.5806451613,1178.3836956522,...,494.4347826087,366.6774193548,435.5333333333,485.0000000000,519.4000000000,3000.0508474576,2886.3220338983,2147.6101694915,3248.5932203390,0
8,GWANGGYO,189.2950819672,194.6721311475,160.5737704918,242.9672131148,420.9677419355,463.0645161290,394.0967741935,507.7741935484,232.5652173913,...,321.3619565217,90.1935483871,197.3000000000,330.5000000000,225.9333333333,565.6949152542,581.1016949153,486.7627118644,641.2033898305,0
9,SUWON,627.7540983607,712.5245901639,513.0655737705,833.9508196721,1598.1935483871,1592.6129032258,1181.9032258065,1773.6129032258,696.0978260870,...,271.1956521739,182.2580645161,254.0000000000,217.4333333333,325.4666666667,1768.1694915254,1862.5084745763,1634.1864406780,2221.0508474576,0


In [13]:
df_region_summary_total.columns

Index(['index', '2019-2-0', '2019-2-1', '2019-2-2', '2019-2-3', '2019-3-0',
       '2019-3-1', '2019-3-2', '2019-3-3', '2020-0-0', '2020-0-1', '2020-0-2',
       '2020-0-3', '2020-1-0', '2020-1-1', '2020-1-2', '2020-1-3', '2020-2-0',
       '2020-2-1', '2020-2-2', '2020-2-3', '2020-3-0', '2020-3-1', '2020-3-2',
       '2020-3-3', '2021-0-0', '2021-0-1', '2021-0-2', '2021-0-3', '2021-1-0',
       '2021-1-1', '2021-1-2', '2021-1-3', '2021-2-0', '2021-2-1', '2021-2-2',
       '2021-2-3', '2021-3-0', '2021-3-1', '2021-3-2', '2021-3-3', '2022-0-0',
       '2022-0-1', '2022-0-2', '2022-0-3', '2022-1-0', '2022-1-1', '2022-1-2',
       '2022-1-3', '2022-2-0', '2022-2-1', '2022-2-2', '2022-2-3', '2022-3-0',
       '2022-3-1', '2022-3-2', '2022-3-3', 'target'],
      dtype='string', name='idx')

In [14]:
X_2019 = df_region_summary_total[['2019-2-0', '2019-2-1', '2019-2-2', '2019-2-3'
                                  , '2019-3-0', '2019-3-1', '2019-3-2', '2019-3-3']]
X_2020 = df_region_summary_total[['2020-0-0', '2020-0-1', '2020-0-2', '2020-0-3'
                                  , '2020-1-0', '2020-1-1', '2020-1-2', '2020-1-3'
                                  , '2020-2-0', '2020-2-1', '2020-2-2', '2020-2-3'
                                  , '2020-3-0', '2020-3-1', '2020-3-2', '2020-3-3']]

y = df_region_summary_total[['index']]

In [1]:
## 하지마 겁나 오래걸림
# sns.pairplot(df_region_summary_total, hue = "target")
# plt.show()